In [14]:
import pyspark
import os
import json
import argparse

from dotenv import load_dotenv
from pathlib import Path
from pyspark.sql.types import StructType
from pyspark.sql.functions import to_timestamp,col,when

## Initialization

In [15]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

False

In [16]:
postgres_host = os.getenv('POSTGRES_CONTAINER_NAME')
postgres_db = os.getenv('POSTGRES_DB')
postgres_user = os.getenv('POSTGRES_USER')
postgres_password = os.getenv('POSTGRES_PASSWORD')

In [17]:
sparkcontext = pyspark.SparkContext.getOrCreate(conf=(
        pyspark
        .SparkConf()
        .setAppName('Dibimbing')
        .setMaster('local')
        .set("spark.jars", "/opt/postgresql-42.2.18.jar")
    ))
sparkcontext.setLogLevel("WARN")

spark = pyspark.sql.SparkSession(sparkcontext.getOrCreate())

In [18]:
spark

## UDF

In [31]:
import pyspark
from pyspark.sql import SparkSession

spark_host = "spark://dibimbing-dataeng-spark-master:7077"

# Create SparkSession with necessary configurations
spark = SparkSession.builder \
    .appName('Dibimbing') \
    .master(spark_host) \
    .config("spark.jars", "/path/to/postgresql-42.2.18.jar") \
    .getOrCreate()

# Define connection properties
postgres_url = "jdbc:postgresql://localhost:5432/warehouse"
properties = {
    "user": "user",
    "password": "password",
    "driver": "org.postgresql.Driver"
}



In [32]:
retail_df = spark.read.jdbc(
    jdbc_url,
    'public.retail',
    properties=jdbc_properties
)

Py4JJavaError: An error occurred while calling o246.setProperty.
: java.lang.NullPointerException
	at java.base/java.util.concurrent.ConcurrentHashMap.putVal(ConcurrentHashMap.java:1011)
	at java.base/java.util.concurrent.ConcurrentHashMap.put(ConcurrentHashMap.java:1006)
	at java.base/java.util.Properties.put(Properties.java:1334)
	at java.base/java.util.Properties.setProperty(Properties.java:229)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:78)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:567)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.j

In [22]:
# Assuming df is your DataFrame containing the data
# Import necessary libraries
from pyspark.sql import functions as F

# Group by StockCode and perform aggregation
analysis_result = df.groupBy("StockCode") \
                    .agg(
                        F.count("InvoiceNo").alias("TotalTransactions"),
                        F.sum("Quantity").alias("TotalQuantity"),
                        F.avg("UnitPrice").alias("AverageUnitPrice")
                    ) \
                    .orderBy("TotalTransactions", ascending=False)

# Show the result
analysis_result.show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `StockCode` cannot be resolved. Did you mean one of the following? [`id`, `fruit`].;
'Aggregate ['StockCode], ['StockCode, count('InvoiceNo) AS TotalTransactions#142, sum('Quantity) AS TotalQuantity#144, avg('UnitPrice) AS AverageUnitPrice#146]
+- LogicalRDD [id#0L, fruit#1], false


In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
pandas_df = analysis_result.toPandas()

# Save Pandas DataFrame to CSV file
csv_path = "/path/to/analysis_result.csv"
pandas_df.to_csv(csv_path, index=False)

